In [1]:
import numpy as np
import pandas as pd
import math
import torch

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import seaborn as sns

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from PIL import Image
from torchvision import transforms, datasets

import torch.nn.functional as F
import torch.nn as nn

In [2]:
california = fetch_california_housing()
feature_names = california.feature_names
data = california.data
df = pd.DataFrame(data, columns=feature_names)
df['target'] = california.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [3]:
df.shape

(20640, 9)

In [4]:
X = df.drop(['target'], axis=1)
y = df['target']

In [5]:
# Масштабирование
X_sc = StandardScaler().fit(X).transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size = 0.25, random_state=13)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15480, 8)
(5160, 8)
(15480,)
(5160,)


In [7]:
class MyCalifornia(torch.utils.data.Dataset):
    
    def __init__(self,df_train, y_train):
        
        df_train = np.array(df_train)
        y_train = np.array(y_train)
        self.x=torch.from_numpy(df_train)
        self.y = torch.from_numpy(y_train)
        self.n_samples= df_train.shape[0]
        
    
    def __getitem__(self, index):
        
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples
    

In [8]:
dataset =  MyCalifornia(X_train, y_train)

In [9]:
first_data = dataset[0]
first_data

(tensor([-0.1860,  0.5849, -0.3561, -0.0049, -0.0605, -0.0968, -0.6657,  0.6687],
        dtype=torch.float64),
 tensor(2.6800, dtype=torch.float64))

In [10]:
features, labels = first_data
print(features, labels)

tensor([-0.1860,  0.5849, -0.3561, -0.0049, -0.0605, -0.0968, -0.6657,  0.6687],
       dtype=torch.float64) tensor(2.6800, dtype=torch.float64)


In [11]:
train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=4,
                          shuffle=True, num_workers=2)

In [20]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)
        return x

In [21]:
net = FeedForward(8, 200)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [22]:
net

FeedForward(
  (fc1): Perceptron(
    (fc): Linear(in_features=8, out_features=200, bias=True)
  )
  (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
  (fc2): Perceptron(
    (fc): Linear(in_features=200, out_features=1, bias=True)
  )
)

In [24]:
num_epochs = 10

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


RuntimeError: expected scalar type Float but found Double

In [ ]:
# num_epochs = 10

# for epoch in range(num_epochs):  
#     running_loss, running_items, running_right = 0.0, 0.0, 0.0
#     for i, data in enumerate(train_loader):
#         inputs, labels = data[0], data[1]

#         # обнуляем градиент
#         optimizer.zero_grad()

#         outputs = net(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # выводим статистику о процессе обучения
#         running_loss += loss.item()
#         running_items += len(labels)
#         running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
#         # выводим статистику о процессе обучения
#         if i % 150 == 0:    # печатаем каждые 300 mini-batches
#             net.eval()
            
#             print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
#                   f'Step [{i + 1}/{len(train_loader)}]. ' \
#                   f'Loss: {running_loss / running_items:.3f}. ' \
#                   f'Acc: {running_right / running_items:.3f}', end='. ')
#             running_loss, running_items, running_right = 0.0, 0.0, 0.0

#             test_running_right, test_running_total = 0.0, 0.0
#             for i, data in enumerate(test_loader):
            
#                 test_outputs = net(data[0])
#                 test_running_total += len(data[1])
#                 test_running_right += (data[1] == torch.max(test_outputs, 1)[1]).sum()
            
#             print(f'Test acc: {test_running_right / test_running_total:.3f}')
#             net.train()
        
# print('Training is finished!')